In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import json
import numpy 
import os
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline  
import pyarrow
import sys

from datetime import date
from dotenv import load_dotenv
from sqlalchemy import create_engine
from os import path
from typing import List,Dict, Tuple
from collections import defaultdict
pd.set_option("display.max_columns", None)

load_dotenv(verbose=True)
BIGQUERY_CREDENTIALS_PATH = os.environ.get('BIGQUERY_CREDENTIALS_PATH')
engine = create_engine('bigquery://bespoke-financial/ProdMetrcData', credentials_path=os.path.expanduser(BIGQUERY_CREDENTIALS_PATH))

sys.path.append(path.realpath(path.join(os.getcwd(), "../core")))
sys.path.append(path.realpath(path.join(os.getcwd(), "../../scripts")))
sys.path.append(path.realpath(path.join(os.getcwd(), "../../src")))

from underwriting import inventory_freshness_analyzer as ifa
from underwriting import inventory_freshness_analysis_query as ifa_query
import create_queries
import prepare_data

from bespoke.inventory.analysis.shared import download_util, inventory_types
from bespoke.inventory.analysis import active_inventory_util as util
from bespoke.inventory.analysis import inventory_valuations_util as valuations_util

%load_ext autoreload
%autoreload 2

In [3]:
TRANSFER_PACKAGES_START_DATE = '2020-01-01'
SALES_TRANSACTIONS_START_DATE = '2020-01-01'
SALES_TRANSACTIONS_END_DATE = '2023-01-01'
today = date.today()

# Analyzer

In [4]:
analyzer = ifa.Analyzer()

In [5]:
# Updating the common attributes used for pulling query information
analyzer.update_class_attributes(['EL'], ['C9-0000370-LIC'], True)

In [6]:
# Find top ranked products/product categories by customized order column
analyzer.find_most_valuable_products('tx_product_name', 'avg_days_since_sale', 'ASC', 10)

,tx_product_name,avg_days_since_sale,max_days_since_sale,min_days_since_sale,number_of_sales,quantity_sold,avg_sale_price
0,HUMBOLDT TREES - Flower - LITTLE TREES - 3.5g ...,3.928571,7,0,14,20.0,30.000000
1,Kings Garden GMO Flower Prepackaged Quarter*,4.866667,10,0,15,16.0,95.000000
2,Lifted Organics American Weed Sour Strawberry ...,6.000000,13,0,24,24.0,130.000000
3,Highrize - OG Kush Premium Flower Single Tubes,6.240000,10,2,25,25.0,1.000000
4,XJ-13 1/4 oz Jar,6.812500,22,1,16,16.0,80.000000
5,Peach Ozz-Baker's 1g Pre-roll,6.966667,22,2,30,30.0,0.996667
6,Flower Jars - Peanut Butter Breath - 3.5g,7.181818,15,0,22,28.0,42.795455
7,Wedding Cake LR Liquid Diamonds 1g 510 Vape Ca...,7.583333,15,1,12,12.0,45.000000
8,Kings Garden Cake Walk Live Budder (1g),8.250000,14,0,12,12.0,35.000000
9,HUMBOLDT TREES - Pre Roll Single - 0.7g - Tropaya,8.576923,17,1,26,32.0,9.961538


In [7]:
# Find top ranked product names categories by sale velocity (Without quantity normalization)
analyzer.find_most_valuable_products_by_velocity_sales_weighted('tx_product_name', 15)

,tx_product_name,avg_days_since_sale,max_days_since_sale,min_days_since_sale,number_of_sales,quantity_sold,avg_sale_price,sale_velocity_per_day,quantity_sale_velocity_per_day
0,Surplus: 1g Cart: Blueberry Cookies [I],27.652174,59,5,138,148.0,39.913043,4.990566,5.352201
1,Surplus: 1g Cart: Trainwreck [H],32.372671,92,3,161,175.0,39.751553,4.973331,5.405794
2,Peach Ozz-Baker's 1g Pre-roll,6.966667,22,2,30,30.0,0.996667,4.306220,4.306220
3,Highrize Sour Patch Kidz PF .7g,17.402778,35,1,72,175.0,4.986111,4.137271,10.055866
4,Highrize - OG Kush Premium Flower Single Tubes,6.240000,10,2,25,25.0,1.000000,4.006410,4.006410
5,Lifted Organics American Weed Sour Strawberry ...,6.000000,13,0,24,24.0,130.000000,4.000000,4.000000
6,"Buddies THC 50mg Capsule 20pc, Hashtag Distrib...",21.612500,51,1,80,94.0,55.000000,3.701562,4.349335
7,Surplus: 1g Cart: Purple Lemonade [S],29.333333,70,3,108,125.0,40.000000,3.681818,4.261364
8,HUMBOLDT TREES - Flower - LITTLE TREES - 3.5g ...,3.928571,7,0,14,20.0,30.000000,3.563636,5.090909
9,Calm 108-1g Pre-roll,8.870968,21,2,31,32.0,1.000000,3.494545,3.607273


In [8]:
# Find top ranked product names by sale velocity (With quantity normalization)
analyzer.find_most_valuable_products_by_velocity_sales_weighted('tx_product_name', 15, True)

,tx_product_name,avg_days_since_sale,max_days_since_sale,min_days_since_sale,number_of_sales,quantity_sold,avg_sale_price,sale_velocity_per_day,quantity_sale_velocity_per_day
0,Highrize Sour Patch Kidz PF .7g,17.402778,35,1,72,175.0,4.986111,4.137271,10.055866
1,Surplus: 1g Cart: Trainwreck [H],32.372671,92,3,161,175.0,39.751553,4.973331,5.405794
2,Surplus: 1g Cart: Blueberry Cookies [I],27.652174,59,5,138,148.0,39.913043,4.990566,5.352201
3,HUMBOLDT TREES - Flower - LITTLE TREES - 3.5g ...,3.928571,7,0,14,20.0,30.000000,3.563636,5.090909
4,Surplus: 1g Cart: Green Crack [S],39.838235,87,4,136,198.0,39.161029,3.413806,4.970100
5,Kings Garden GMO Flower Prepackaged Gram*,10.041667,26,1,24,48.0,16.995833,2.390041,4.780083
6,Kings Garden Wedding Gushers Flower Prepackage...,14.225806,36,1,31,68.0,15.379032,2.179138,4.780045
7,"Buddies THC 50mg Capsule 20pc, Hashtag Distrib...",21.612500,51,1,80,94.0,55.000000,3.701562,4.349335
8,Peach Ozz-Baker's 1g Pre-roll,6.966667,22,2,30,30.0,0.996667,4.306220,4.306220
9,Surplus: 1g Cart: Purple Lemonade [S],29.333333,70,3,108,125.0,40.000000,3.681818,4.261364


In [47]:
# Find top ranked product category names categories by sale velocity (Without quantity normalization)
analyzer.find_most_valuable_products_by_velocity_sales_weighted('tx_product_category_name, tx_product_name', 15)

,tx_product_category_name,tx_product_name,avg_days_since_sale,max_days_since_sale,min_days_since_sale,number_of_sales,quantity_sold,avg_cost,avg_sale_price,sale_velocity_per_day,quantity_sale_velocity_per_day
0,Vape Cartridge (volume - each),Surplus: 1g Cart: Blueberry Cookies [I],27.652174,59,5,138,148.0,16.500000,39.913043,4.990566,5.352201
1,Vape Cartridge (volume - each),Surplus: 1g Cart: Trainwreck [H],32.372671,92,3,161,175.0,16.500000,39.751553,4.973331,5.405794
2,Pre-Roll Leaf,Peach Ozz-Baker's 1g Pre-roll,6.966667,22,2,30,30.0,0.010000,0.996667,4.306220,4.306220
3,Pre-Roll Flower,Highrize Sour Patch Kidz PF .7g,17.402778,35,1,72,175.0,0.725000,4.986111,4.137271,10.055866
4,Pre-Roll Flower,Highrize - OG Kush Premium Flower Single Tubes,6.240000,10,2,25,25.0,1.000000,1.000000,4.006410,4.006410
5,Flower (packaged half ounce - each),Lifted Organics American Weed Sour Strawberry ...,6.000000,13,0,24,24.0,42.000000,130.000000,4.000000,4.000000
6,Capsule (weight - each),"Buddies THC 50mg Capsule 20pc, Hashtag Distrib...",21.612500,51,1,80,94.0,21.000000,55.000000,3.701562,4.349335
7,Flower (packaged eighth - each),HUMBOLDT TREES - Flower - LITTLE TREES - 3.5g ...,3.928571,7,0,14,20.0,11.500000,30.000000,3.563636,5.090909
8,Pre-Roll Flower,Calm 108-1g Pre-roll,8.870968,21,2,31,32.0,0.020000,1.000000,3.494545,3.607273
9,Pre-Roll Flower,1.0g Cone - Mendo Breath,35.264463,169,1,121,125.0,0.010000,6.113388,3.431216,3.544645


In [10]:
# Find top ranked product category names categories by sale velocity (With quantity normalization)
analyzer.find_most_valuable_products_by_velocity_sales_weighted('tx_product_category_name, tx_product_name', 15, True)

,tx_product_category_name,tx_product_name,avg_days_since_sale,max_days_since_sale,min_days_since_sale,number_of_sales,quantity_sold,avg_sale_price,sale_velocity_per_day,quantity_sale_velocity_per_day
0,Pre-Roll Flower,Highrize Sour Patch Kidz PF .7g,17.402778,35,1,72,175.0,4.986111,4.137271,10.055866
1,Vape Cartridge (volume - each),Surplus: 1g Cart: Trainwreck [H],32.372671,92,3,161,175.0,39.751553,4.973331,5.405794
2,Vape Cartridge (volume - each),Surplus: 1g Cart: Blueberry Cookies [I],27.652174,59,5,138,148.0,39.913043,4.990566,5.352201
3,Flower (packaged eighth - each),HUMBOLDT TREES - Flower - LITTLE TREES - 3.5g ...,3.928571,7,0,14,20.0,30.000000,3.563636,5.090909
4,Vape Cartridge (volume - each),Surplus: 1g Cart: Green Crack [S],39.838235,87,4,136,198.0,39.161029,3.413806,4.970100
5,Flower (packaged gram - each),Kings Garden GMO Flower Prepackaged Gram*,10.041667,26,1,24,48.0,16.995833,2.390041,4.780083
6,Flower (packaged gram - each),Kings Garden Wedding Gushers Flower Prepackage...,14.225806,36,1,31,68.0,15.379032,2.179138,4.780045
7,Capsule (weight - each),"Buddies THC 50mg Capsule 20pc, Hashtag Distrib...",21.612500,51,1,80,94.0,55.000000,3.701562,4.349335
8,Pre-Roll Leaf,Peach Ozz-Baker's 1g Pre-roll,6.966667,22,2,30,30.0,0.996667,4.306220,4.306220
9,Pre-Roll Flower,Highrize - OG Kush Premium Flower Single Tubes,6.240000,10,2,25,25.0,1.000000,4.006410,4.006410


In [11]:
# Output data for inventory valuation calculation using Inventory-Sales joined data 
df_v1 = analyzer.get_inventory_valuation_query('last_sale_valuation', by_product_name=True, discount_rate=.2)
df_v1

,package_id,price_per_unit,quantity,product_name,product_category_name,inventory_year_diff,last_sold_price_valuation,day_sold_rank
0,11497141,25.60,8.0,Sugar Cone - |GLAZE| Applicator (1.0 g) Glazed...,Other Concentrate (weight - each),1.191781,204.80,1
1,12801666,20.48,34.0,Level Protab Hangover,Extract (weight - each),0.846575,696.32,1
2,12801667,12.80,8.0,Petra Mints Pineapple - 100mg,Edible (weight - each),0.849315,102.40,1
3,12801668,22.40,36.0,Level Tablingual Calm CBG,Extract (weight - each),0.953425,806.40,1
4,12801670,19.20,33.0,Level Tablingual Sleep CBN,Extract (weight - each),0.934247,633.60,1
...,...,...,...,...,...,...,...,...
281,8164573,21.00,0.0,BIIIT - Sour Straws Assorted,Edible (weight - each),0.320548,0.00,1
282,8164574,21.00,0.0,BIIIT - Sour Straws Assorted,Edible (weight - each),0.320548,0.00,1
283,8486461,23.04,4.0,Fiori Friend CBD Pet Supplement (300mg CBD),Tincture (volume - each),1.405479,92.16,1
284,9166114,64.00,1.0,N5 - Harmony 30ml - 30ml,Tincture (volume - each),1.358904,64.00,1


In [12]:
# Output data for inventory valuation calculation using Inventory-Sales joined data 
df_v2 = analyzer.get_inventory_valuation_query('discount_valuation', by_product_name=True, discount_rate=.2)
df_v2

,discounted_price,package_id,product_name,product_category_name,quantity,inventory_year_diff,price_per_unit
0,26.420454,11497141,Sugar Cone - |GLAZE| Applicator (1.0 g) Glazed...,Other Concentrate (weight - each),8.0,1.191781,32.832727
1,30.348268,12801666,Level Protab Hangover,Extract (weight - each),34.0,0.846575,35.413333
2,16.399981,12801667,Petra Mints Pineapple - 100mg,Edible (weight - each),8.0,0.849315,19.146667
3,20.973177,12801668,Level Tablingual Calm CBG,Extract (weight - each),36.0,0.953425,24.955000
4,21.585000,12801670,Level Tablingual Sleep CBN,Extract (weight - each),33.0,0.934247,25.593333
...,...,...,...,...,...,...,...
281,21.505694,8164573,BIIIT - Sour Straws Assorted,Edible (weight - each),0.0,0.320548,22.800000
282,21.505694,8164574,BIIIT - Sour Straws Assorted,Edible (weight - each),0.0,0.320548,22.800000
283,27.095957,8486461,Fiori Friend CBD Pet Supplement (300mg CBD),Tincture (volume - each),4.0,1.405479,35.010000
284,54.118077,9166114,N5 - Harmony 30ml - 30ml,Tincture (volume - each),1.0,1.358904,69.333333


In [13]:
# Output data and inventory valuation using Inventory-Sales joined data and last sale price method 
df_v1, valuation_1 = analyzer.find_inventory_valuation('last_sale_valuation', by_product_name=True, discount_rate=.2)
df_v1.head()

,package_id,price_per_unit,quantity,product_name,product_category_name,inventory_year_diff,last_sold_price_valuation,day_sold_rank,product_type,shelf_life,expired
0,11497141,0.00,8.0,Sugar Cone - |GLAZE| Applicator (1.0 g) Glazed...,Other Concentrate (weight - each),1.191781,204.80,1,Concentrates,1.0,True
1,12801666,20.48,34.0,Level Protab Hangover,Extract (weight - each),0.846575,696.32,1,Concentrates,1.0,False
2,12801667,12.80,8.0,Petra Mints Pineapple - 100mg,Edible (weight - each),0.849315,102.40,1,Edibles,1.0,False
3,12801668,22.40,36.0,Level Tablingual Calm CBG,Extract (weight - each),0.953425,806.40,1,Concentrates,1.0,False
4,12801670,19.20,33.0,Level Tablingual Sleep CBN,Extract (weight - each),0.934247,633.60,1,Concentrates,1.0,False


In [14]:
valuation_1

95065.68

In [15]:
# Output data and inventory valuation using Inventory-Sales joined data and discount valuation method 
df_v2, valuation_2 = analyzer.find_inventory_valuation('discount_valuation', by_product_name=True, discount_rate=.2)
df_v2.head()

,discounted_price,package_id,product_name,product_category_name,quantity,inventory_year_diff,price_per_unit,product_type,shelf_life,expired
0,0.000000,11497141,Sugar Cone - |GLAZE| Applicator (1.0 g) Glazed...,Other Concentrate (weight - each),8.0,1.191781,32.832727,Concentrates,1.0,True
1,30.348268,12801666,Level Protab Hangover,Extract (weight - each),34.0,0.846575,35.413333,Concentrates,1.0,False
2,16.399981,12801667,Petra Mints Pineapple - 100mg,Edible (weight - each),8.0,0.849315,19.146667,Edibles,1.0,False
3,20.973177,12801668,Level Tablingual Calm CBG,Extract (weight - each),36.0,0.953425,24.955000,Concentrates,1.0,False
4,21.585000,12801670,Level Tablingual Sleep CBN,Extract (weight - each),33.0,0.934247,25.593333,Concentrates,1.0,False


In [17]:
valuation_2

93426.38496919227

In [18]:
# Output data and inventory valuation using Inventory-Incoming joined data and last sale price method 
df_i1, valuation = analyzer.find_incoming_inventory_valuation('discount_valuation', discount_rate=.2)
df_i1.head()

,discounted_price,package_id,product_name,product_category_name,quantity,inventory_year_diff,price_per_unit,product_type,shelf_life,expired
0,0.000000,11497141,Sugar Cone - |GLAZE| Applicator (1.0 g) Glazed...,Other Concentrate (weight - each),8.0,1.191781,15.0,Concentrates,1.0,True
1,11.140648,12801666,Level Protab Hangover,Extract (weight - each),34.0,0.846575,13.0,Concentrates,1.0,False
2,6.852360,12801667,Petra Mints Pineapple - 100mg,Edible (weight - each),8.0,0.849315,8.0,Edibles,1.0,False
3,10.085278,12801668,Level Tablingual Calm CBG,Extract (weight - each),36.0,0.953425,12.0,Concentrates,1.0,False
4,10.120604,12801670,Level Tablingual Sleep CBN,Extract (weight - each),33.0,0.934247,12.0,Concentrates,1.0,False


In [19]:
valuation

44311.11100094838

In [20]:
# Output data and inventory valuation using Inventory-Incoming joined data, joined on package_id and last sale price method 
df_i1, valuation = analyzer.find_incoming_inventory_valuation('discount_valuation', False, discount_rate=.2)
df_i1.head()

,discounted_price,package_id,product_name,product_category_name,quantity,inventory_year_diff,price_per_unit,product_type,shelf_life,expired
0,0.000000,11497141,Sugar Cone - |GLAZE| Applicator (1.0 g) Glazed...,Other Concentrate (weight - each),8.0,1.191781,15.0,Concentrates,1.0,True
1,11.140648,12801666,Level Protab Hangover,Extract (weight - each),34.0,0.846575,13.0,Concentrates,1.0,False
2,6.852360,12801667,Petra Mints Pineapple - 100mg,Edible (weight - each),8.0,0.849315,8.0,Edibles,1.0,False
3,10.085278,12801668,Level Tablingual Calm CBG,Extract (weight - each),36.0,0.953425,12.0,Concentrates,1.0,False
4,10.120604,12801670,Level Tablingual Sleep CBN,Extract (weight - each),33.0,0.934247,12.0,Concentrates,1.0,False


In [21]:
valuation

44576.099563279

In [23]:
# Find inventory valuation using all available methods using data that is joined by package_id
analyzer.find_all_valuation(False, discount_rate=.2)


#### Inventory Valuation based on sales data using last sale method: $85384.0 ####
#### Inventory Valuation based on sales data using discount time method: $83506.0 ####
#### Inventory Valuation based on incoming data using discount time method: $44576.0 ####


(85384.0, 83506.0, 44576.0)

In [24]:
# Find inventory valuation using all available methods using data that is joined by product_name
analyzer.find_all_valuation(True, discount_rate=.2)


#### Inventory Valuation based on sales data using last sale method: $95066.0 ####
#### Inventory Valuation based on sales data using discount time method: $93426.0 ####
#### Inventory Valuation based on incoming data using discount time method: $44311.0 ####


(95066.0, 93426.0, 44311.0)

In [29]:
# Find product highest velocity product names with a specific product category
analyzer.find_most_valuable_products_by_velocity_sales_weighted('tx_product_category_name, tx_product_name', 15, True, 'Pre-Roll Flower')

,tx_product_category_name,tx_product_name,avg_days_since_sale,max_days_since_sale,min_days_since_sale,number_of_sales,quantity_sold,avg_sale_price,sale_velocity_per_day,quantity_sale_velocity_per_day
0,Pre-Roll Flower,Highrize Sour Patch Kidz PF .7g,17.402778,35,1,72,175.0,4.986111,4.137271,10.055866
1,Pre-Roll Flower,Highrize - OG Kush Premium Flower Single Tubes,6.240000,10,2,25,25.0,1.000000,4.006410,4.006410
2,Pre-Roll Flower,HUMBOLDT TREES - Pre Roll Single - 0.7g - Tropaya,8.576923,17,1,26,32.0,9.961538,3.031390,3.730942
3,Pre-Roll Flower,HUMBOLDT TREES - Pre Roll Single - 0.7g - Flow...,17.185185,49,1,54,64.0,4.666667,3.142241,3.724138
4,Pre-Roll Flower,Calm 108-1g Pre-roll,8.870968,21,2,31,32.0,1.000000,3.494545,3.607273
5,Pre-Roll Flower,1.0g Cone - Mendo Breath,35.264463,169,1,121,125.0,6.113388,3.431216,3.544645
6,Pre-Roll Flower,LIIIT - Pink Acai Pre-roll - 0.3g,12.117647,27,4,17,36.0,4.926471,1.402913,2.970874
7,Pre-Roll Flower,HUMBOLDT TREES - Pre Roll Single - 0.7g - Slur...,11.258065,22,1,31,32.0,9.967742,2.753582,2.842407
8,Pre-Roll Flower,LIIIT - Stiiizy OG Pre-roll - 0.3g,12.800000,27,5,20,36.0,4.975000,1.562500,2.812500
9,Pre-Roll Flower,HUMBOLDT TREES - Pre Roll Single - 0.7g - Key ...,13.117647,21,5,17,32.0,9.941176,1.295964,2.439462


In [30]:
# Find product highest velocity product names with a specific product category
analyzer.find_most_valuable_products_by_velocity_sales_weighted('tx_product_category_name, tx_product_name', 15, True, 'Flower (packaged eighth - each)')

,tx_product_category_name,tx_product_name,avg_days_since_sale,max_days_since_sale,min_days_since_sale,number_of_sales,quantity_sold,avg_sale_price,sale_velocity_per_day,quantity_sale_velocity_per_day
0,Flower (packaged eighth - each),HUMBOLDT TREES - Flower - LITTLE TREES - 3.5g ...,3.928571,7,0,14,20.0,30.000000,3.563636,5.090909
1,Flower (packaged eighth - each),Flower Jars - Peanut Butter Breath - 3.5g,7.181818,15,0,22,28.0,42.795455,3.063291,3.898734
2,Flower (packaged eighth - each),HUMBOLDT TREES - Flower - LITTLE TREES - 3.5g ...,15.509434,37,0,53,60.0,31.698113,3.417275,3.868613
3,Flower (packaged eighth - each),"LEGION - Flower - 3.5g - CPG ""Bigs"" Indoor - C...",17.666667,39,0,51,64.0,49.607843,2.886792,3.622642
4,Flower (packaged eighth - each),HUMBOLDT TREES - Flower - LITTLE TREES - 3.5g ...,15.029412,41,0,34,52.0,33.088235,2.262231,3.459883
5,Flower (packaged eighth - each),HUMBOLDT TREES - Flower - LITTLE TREES - 3.5g ...,17.704545,34,2,44,55.0,30.000000,2.485237,3.106547
6,Flower (packaged eighth - each),Almora Farm: 3.5g Flower: Sweet Diesel [S],11.944444,21,5,18,32.0,24.861111,1.506977,2.679070
7,Flower (packaged eighth - each),Papa's Herb Slurricane Flower Prepackaged 1/8th,10.157895,20,1,19,24.0,25.000000,1.870466,2.362694
8,Flower (packaged eighth - each),HUMBOLDT TREES - Flower - LITTLE TREES - 3.5g ...,27.800000,76,1,55,64.0,29.890909,1.978417,2.302158
9,Flower (packaged eighth - each),Kings Garden Kings Cake Flower Prepackaged 1/8th*,27.833333,55,0,54,64.0,48.750000,1.940120,2.299401


In [49]:
# Find product highest velocity product names with a specific product category
analyzer.find_most_valuable_products_by_velocity_sales_weighted('tx_product_category_name, tx_product_name', 15, True, 'Capsule (weight - each)')

,tx_product_category_name,tx_product_name,avg_days_since_sale,max_days_since_sale,min_days_since_sale,number_of_sales,quantity_sold,avg_cost,avg_sale_price,sale_velocity_per_day,quantity_sale_velocity_per_day
0,Capsule (weight - each),"Buddies THC 50mg Capsule 20pc, Hashtag Distrib...",21.612500,51,1,80,94.0,21.00,55.000000,3.701562,4.349335
1,Capsule (weight - each),"Buddies THC 25mg Capsule 40pc, Hashtag Distrib...",27.083333,78,1,60,65.0,21.00,60.000000,2.215385,2.400000
2,Capsule (weight - each),"Buddies THC 100mg Capsule 10pc, Hashtag Distri...",52.461538,116,1,104,123.0,24.00,64.812500,1.982405,2.344575
3,Capsule (weight - each),Buddies - Capsule - 50mg THC 20pc,18.576923,44,2,26,36.0,21.00,54.365385,1.399586,1.937888
4,Capsule (weight - each),Buddies - Capsule - 100mg THC 10pc,26.266667,51,2,15,24.0,24.00,64.566667,0.571066,0.913706
5,Capsule (weight - each),"Buddies THC Gel Capsule 50mg 20pc, Hashtag Dis...",33.388889,51,6,18,24.0,21.00,54.694444,0.539101,0.718802
6,Capsule (weight - each),"Buddies THC 10mg Capsule 60pc, Hashtag Distrib...",75.152174,123,3,46,51.0,16.00,50.000000,0.612091,0.678623
7,Capsule (weight - each),"Buddies LD 25mg Indica Capsule 40pc, Hashtag D...",49.640000,76,1,25,26.0,30.00,65.000000,0.503626,0.523771
8,Capsule (weight - each),15824 Kikoko Tablet Indica X 30mg Single Table...,24.250000,34,13,12,12.0,0.01,1.000000,0.494845,0.494845
9,Capsule (weight - each),"Buddies LD 25mg Sativa Capsule 40pc, Hashtag D...",47.733333,88,1,15,17.0,30.00,66.000000,0.314246,0.356145


In [32]:
analyzer.top_product_and_product_category_data_generation(15, True)

(               tx_product_category_name  avg_days_since_sale  \
 0        Vape Cartridge (weight - each)            38.124295   
 1       Flower (packaged eighth - each)            45.486741   
 2        Vape Cartridge (volume - each)            37.453433   
 3                Edible (weight - each)            71.899034   
 4                       Pre-Roll Flower            54.631099   
 5      Flower (packaged quarter - each)            40.425459   
 6               Extract (weight - each)            66.102155   
 7                      Pre-Roll Infused            44.985770   
 8   Flower (packaged half ounce - each)            28.689805   
 9                         Pre-Roll Leaf            35.625000   
 10        Flower (packaged gram - each)            44.643750   
 11              Capsule (weight - each)            47.994152   
 12             Flower (packaged - each)            65.517028   
 13    Other Concentrate (weight - each)            94.845972   
 14             Tincture 

In [45]:
analyzer.top_product_and_product_category_report_generation(15, True)

### Following are the top product categories by sales velocity ###
1. Category: Vape Cartridge (weight - each), 
 Sale velocity: 142.56
2. Category: Flower (packaged eighth - each), 
 Sale velocity: 91.5
3. Category: Vape Cartridge (volume - each), 
 Sale velocity: 74.76
4. Category: Edible (weight - each), 
 Sale velocity: 70.39
5. Category: Pre-Roll Flower, 
 Sale velocity: 41.83
6. Category: Flower (packaged quarter - each), 
 Sale velocity: 23.72
7. Category: Extract (weight - each), 
 Sale velocity: 21.95
8. Category: Pre-Roll Infused, 
 Sale velocity: 20.43
9. Category: Flower (packaged half ounce - each), 
 Sale velocity: 16.38
10. Category: Pre-Roll Leaf, 
 Sale velocity: 16.08
11. Category: Flower (packaged gram - each), 
 Sale velocity: 15.34
12. Category: Capsule (weight - each), 
 Sale velocity: 12.44
13. Category: Flower (packaged - each), 
 Sale velocity: 5.53
14. Category: Other Concentrate (weight - each), 
 Sale velocity: 4.98
15. Category: Tincture (volume - each), 
 

In [51]:
# Use sales margin as a measure to rank the categories
analyzer.find_most_valuable_products_by_margin_weighted('tx_product_category_name', 15)


,tx_product_category_name,avg_days_since_sale,max_days_since_sale,min_days_since_sale,number_of_sales,quantity_sold,avg_cost,avg_sale_price,sales_margin,sale_velocity_per_day,quantity_sale_velocity_per_day
0,Shake (Packaged Half Ounce - each),71.150943,184,7,53,56.0,20.397358,79.584906,0.743703,0.744895,0.787059
1,Flower (packaged - each),65.517028,319,1,323,362.0,18.868607,65.490712,0.711889,4.930016,5.525281
2,Pre-Roll Flower,54.631099,285,0,1865,2285.0,6.288152,20.111791,0.687340,34.138065,41.825994
3,Flower,67.133333,136,18,30,224.0,1.952381,6.000000,0.674603,0.446872,3.336643
4,Flower (packaged quarter - each),40.425459,179,0,872,959.0,21.695639,66.277466,0.672654,21.570565,23.722675
5,Extract (volume - each),122.071429,275,4,14,14.0,8.563571,25.857143,0.668812,0.114687,0.114687
6,Flower (packaged half ounce - each),28.689805,167,0,461,470.0,38.841714,115.301518,0.663129,16.068426,16.382126
7,Extract (weight - each),66.102155,386,0,1253,1451.0,10.778180,31.817342,0.661248,18.955509,21.950873
8,Other Concentrate (weight - each),94.845972,488,0,422,472.0,10.281209,30.327346,0.660992,4.449319,4.976490
9,Edible (volume - each),62.933333,187,0,45,47.0,5.002444,14.588889,0.657106,0.715042,0.746822


In [56]:
# Use sales margin as a measure to rank the categories and product names within product category
analyzer.find_most_valuable_products_by_margin_weighted('tx_product_category_name, tx_product_name', 15, 'Flower (packaged - each)')


,tx_product_category_name,tx_product_name,avg_days_since_sale,max_days_since_sale,min_days_since_sale,number_of_sales,quantity_sold,avg_cost,avg_sale_price,sales_margin,sale_velocity_per_day,quantity_sale_velocity_per_day
0,Flower (packaged - each),3.5g - Purple Punch,51.466667,120,3,30,38.0,7.220000,44.200000,0.836652,0.582902,0.738342
1,Flower (packaged - each),3.5g - Chocolate Haze,51.444444,97,4,27,32.0,9.000000,54.796296,0.835755,0.524838,0.622030
2,Flower (packaged - each),3.5g - Tahoe OG,93.166667,209,2,30,32.0,9.000000,54.633333,0.835265,0.322004,0.343470
3,Flower (packaged - each),3.5g - Jack Rabbit OG,65.224138,173,1,58,66.0,13.037931,52.853448,0.753319,0.889241,1.011895
4,Flower (packaged - each),HONEYLEAF - 3.5G SATIVA,27.466667,43,2,15,16.0,7.650000,24.833333,0.691946,0.546117,0.582524
5,Flower (packaged - each),HONEYLEAF - 3.5G INDICA,19.500000,34,5,16,16.0,7.650000,24.687500,0.690127,0.820513,0.820513
6,Flower (packaged - each),3.5g - Durban Poison,22.241379,50,4,29,38.0,14.296552,43.827586,0.673800,1.303876,1.708527
7,Flower (packaged - each),3.5g - Pure Kush,135.758621,319,2,29,32.0,18.000000,53.431034,0.663117,0.213614,0.235712
8,Flower (packaged - each),3.5g - Master Kush,139.833333,260,9,30,32.0,18.000000,51.333333,0.649351,0.214541,0.228844


In [57]:
# Use sales margin as a measure to rank the categories and product names within product category
analyzer.find_most_valuable_products_by_margin_weighted('tx_product_category_name, tx_product_name', 15, 'Flower (packaged eighth - each)')


,tx_product_category_name,tx_product_name,avg_days_since_sale,max_days_since_sale,min_days_since_sale,number_of_sales,quantity_sold,avg_cost,avg_sale_price,sales_margin,sale_velocity_per_day,quantity_sale_velocity_per_day
0,Flower (packaged eighth - each),16689 HENRY'S ORIGINAL Flower 3.5g Jar Sativa ...,46.307692,67,20,13,16.0,8.000000,40.000000,0.800000,0.280731,0.345515
1,Flower (packaged eighth - each),16683 HENRY'S ORIGINAL Flower 3.5g Jar Hybrid ...,146.068966,236,2,29,32.0,8.000000,38.220690,0.790689,0.198536,0.219075
2,Flower (packaged eighth - each),Honeyleaf - .Indica - indica - 3.5g,19.269231,33,8,26,28.0,5.950000,24.307692,0.755222,1.349301,1.453094
3,Flower (packaged eighth - each),16868 Pacific Stone Flower 3.5g Pouch Hybrid K...,81.375000,173,7,32,32.0,9.000000,34.890625,0.742051,0.393241,0.393241
4,Flower (packaged eighth - each),FPB150-B8e,35.714286,72,4,21,32.0,8.000000,30.000000,0.733333,0.588000,0.896000
5,Flower (packaged eighth - each),SSC170-B8e,54.540000,97,12,50,64.0,8.000000,29.940000,0.732799,0.916758,1.173451
6,Flower (packaged eighth - each),"LEGION - Flower - 3.5g - CPG ""Bigs"" Indoor - S...",20.000000,34,4,23,32.0,17.000000,60.000000,0.716667,1.150000,1.600000
7,Flower (packaged eighth - each),FloraCal Julius Prepackaged 1/8th,74.851852,124,4,27,32.0,14.660000,49.814815,0.705710,0.360713,0.427511
8,Flower (packaged eighth - each),FloraCal Kush Mints Prepackaged 1/8th*,124.230769,144,71,13,14.0,14.660000,49.230769,0.702219,0.104644,0.112693
9,Flower (packaged eighth - each),FloraCal Lemon Meringue Kush #1 Prepackaged 1/...,23.285714,53,2,28,32.0,15.000000,50.000000,0.700000,1.202454,1.374233
